# Mirsaeid Abolghasemi - Machine Learning final

I ran some cells on different Google accounts and merge them here, because my Colab crashed a lot after running the whole code.


# Alternus Vera 

Course code : **CMPE-257** 
    

    
Name:  **Mirsaeid Abolghasemi**  

-----


### Liar Liar Pants on Fire Dataset Description 
- It has 3 files test, training and valid.
- Each file has 14 columns
    
    Column 1: the ID of the statement ([ID].json).
    
    Column 2: the label.
    
    Column 3: the statement.
    
    Column 4: the subject(s).
    
    Column 5: the speaker.
    
    Column 6: the speaker's job title.
    
    Column 7: the state info.
    
    Column 8: the party affiliation.
    
    Column 9-13: the total credit history count, including the current statement.
    
    Column 14: the context (venue / location of the speech or statement).

### Process of My Approach 
- Load the Data
- Distillation Process
    - Data Cleaning and Text Preprocessing
    - Visualization
- **Feature 1 :** Corpus Structure
- Vector Classification Modeling 
- Ranking and Importance
- Merge all features and individual contributions
- Form Polynomial Equation 
    

### Feature Selection
**List top Features Selected based on research articles**



### The Team Contributions:

|Features  |  Member |
|-----|-----|
| Psychology utility                         |  Varun Bhaseen |  
| Content statistics                 |  Gulya Timokhina | 
| ClickBait                   |  Poornapragna Vadiraj  | 
| Corpus Structure                   |  Mirsaeid Abolghasemi  |   

 
#### Enrichment Dataset Details

- Dataset for our real news data is https://www.kaggle.com/snapcrack/all-the-news#articles1.csv
- Dataset for Fake news data used the Kaggle Fake News Data Set (https://www.kaggle.com/mrisdal/fake-news)

#### Libraries Used 

- NLTK 
- Gensim 
- Numpy
- Pandas
- CSV
- WordCloud
- Seaborn
- Scipy
- Regualr Expression
- Matplotlib
- Sklearn 


#### What did I try and What worked? 

> I tried to add Unigram, Bigrams, Tri-grams, 4-grams from cleaned material for the corpus structure function, vectorizing the documents with TF-IDF, then implementing multiple classification algorithms such as XGBoost, Random Forest, and SVM. Compare for each of the F1 ratings and  accuracies. I have experimented around with the C values transferred in the test, which also makes me minor changes on the accuracy ranking, by vectorizing each text based on ngrams.

#### What did not work?

> While ngrams provided the distinction on the accuracy of the model, however with the main liar-liar dataset, the accuracy of the model did not have so much change, it ranged from 50 percent to 58 percent, while I used various classification algorithms, the accuracy of the model did not increase so much.


#### What alternatives did you try?

> I attempted data amalgamation by collecting extra actual news data and false news from kaggle, cleaning feature material and text columns by adding and equivalent cleaning logic used in the actual data collection, enriching it with the real dataset and feeding it back into the same model.
----

In [ ]:
import pandas as pd
import numpy as np
import csv
#import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import matplotlib.pyplot as plt
from scipy import sparse
# Code source: https://degravek.github.io/project-pages/project1/2017/04/28/New-Notebook/
# Dataset from Chakraborty et al. (https://github.com/bhargaviparanjape/clickbait/tree/master/dataset)

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz

file_id = '1E3-see3z8mcsa8PdAkbx8S-luv6v7pkQ'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile('test.tsv')


file_id = '1ZGVezZTHqFzpsUcSWn-wCuEqYMDUh0oq'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile('train.tsv')

file_id = '1bM-cgvmjW8yhP23EQLgOMRyON8R6w_V5'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile('valid.tsv')


# Content articles1.csv - 50,000 news articles (Articles 1-50,000)
file_id = '1fau5ZGMP47VInKy1V5xfhjID5D9Nq87Y'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile('articles1.csv')


file_id = '1AbayLqq7kxY_UykkLr5zEoJFB8mvT9lS'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile('fake.csv')

In [ ]:
# Read the test, training and valid data from files
# Header = 0 indicates that the first line of the file contains column names,
# As there is no Header, create a column names for each column in the dataset
# delimiter = \t indicates that the fields are seperated by tabs, and 


test_filename = 'test.tsv'
train_filename = 'train.tsv'
valid_filename = 'valid.tsv'

colnames = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 'speakerjobtitle', 'stateinfo','partyaffiliation', 'barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts','context']

train_news = pd.read_csv(train_filename, sep='\t', names = colnames, error_bad_lines=False)
test_news = pd.read_csv(test_filename, sep='\t', names = colnames, error_bad_lines=False)
valid_news = pd.read_csv(valid_filename, sep='\t', names = colnames, error_bad_lines=False)

In [ ]:
# train_news = train_news[:1000]
# test_news = test_news[:1000]

In [ ]:
# Display check the dimensions and the first 2 rows of the file.

print('train dim:',train_news.shape, 'test dim:', test_news.shape)
train_news.iloc[0:2]

## Data Cleaning and Text Preprocessing 

*Steps included in the preprocessing:*
- Remove Special Characters and Punctuations
- Lower case the news
- Tokenization
- Remove Stop Words
- Lemmatization
- Stemming 
- Spell Check 

###  Putting It All Together 

To make the code reusable, we need to create a function that can be called many times.

In [ ]:
import re

def cleaning(raw_news):
    import nltk
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", str(raw_news))
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    # 4. Convert the stopwords list to "set" data type.
    stops = set(nltk.corpus.stopwords.words("english"))
    
    # 5. Remove stop words. 
    words = [w for w in  news_words  if not w in stops]
    
    # 6. Lemmentize 
    wordnet_lem = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    
    # 7. Stemming
    stems = [nltk.stem.SnowballStemmer('english').stem(w) for w in wordnet_lem ]
    
    # 8. Join the stemmed words back into one string separated by space, and return the result.
    return " ".join(stems)

In [ ]:
import nltk
nltk.download('wordnet')

nltk.download('punkt')

#SENT_DETECTOR = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:

import time
# clean training and test data 
# create new column "tokenized"
t1 = time.time()

# Add the processed data to the original data. 
# Perhaps using apply function would be more elegant and concise than using for loop
train_news['clean'] = train_news["headline_text"].apply(cleaning) 

t2 = time.time()
print("\nTime to clean, tokenize and stem train data: \n", len(train_news), "news:", (t2-t1)/60, "min")

t1 = time.time()
test_news['clean'] = test_news["headline_text"].apply(cleaning)

t2 = time.time()
print("\n\nTime to clean, tokenize and stem test data: \n", len(test_news), "news:", (t2-t1)/60, "min")

t1 = time.time()
valid_news['clean'] = valid_news["headline_text"].apply(cleaning)

t2 = time.time()
print("\n\nTime to clean, tokenize and stem valid data: \n", len(valid_news), "news:", (t2-t1)/60, "min")

In [ ]:
# file_id = '1W9Aj0S8Pp8MC13dwbjTSeWPeLn9m-RNP'
# downloaded = drive.CreateFile({'id': file_id})
# #print('Downloaded content "{}"'.format(downloaded.GetContentString()))
# downloaded.GetContentFile('GoogleNews-vectors-negative300.bin.gz')


### [Google News corpus word2vec](http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/)

### Spell Check 

-  You can download the pre-trained model [**here**](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)

- Or clone it from GitHub [**GoogleNews-vectors-negative300**](https://github.com/mmihaltz/word2vec-GoogleNews-vectors)

> It’s 1.5GB! It includes word vectors for a vocabulary of 3 million words and phrases that they trained on roughly 100 billion words from a Google News dataset. The vector length is 300 features.

**3 million words * 300 features * 4bytes/feature = ~3.35GB**

> This file consist of the word2vec -  pre-trained Google News corpus (3 billion running words) to word vector model (3 million 300-dimension English word vectors).

> Look at the [**vocabulory list**](https://github.com/chrisjmccormick/inspect_word2vec/tree/master/vocabulary) used to train this model. Each text file contains 100,000 entries from the model. 


>  There are few things that this dataset contains and not. It has stop words like  “the”, “also”, “should” and does not have stop words like “a”, “and”, “of”. As I have removed the stop words the complexity is reduced as there is no need to check the spelling for stop words. 

> It does have numbers but in the form of entried wiht #. e.g., you won’t find “100”. But it does include entries like “###MHz_DDR2_SDRAM”. 

The model used [**WinPython-64bit-2.7.10.3**](https://winpython.github.io/) for efficient python distribution on Windows system. Helps to run the scripts in batches. 

In [ ]:
# model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
# words = model.index2word

# w_rank = {}
# for i,word in enumerate(words):
#     w_rank[word] = i

# WORDS = w_rank

In [ ]:
# import re
# from collections import Counter

# def words(text): return re.findall(r'\w+', text.lower())

# def P(word, N=sum(WORDS.values())): 
#     "Probability of `word`."
#     return - WORDS.get(word, 0)

# def correction(word): 
#     "Most probable spelling correction for word."
#     return max(candidates(word), key=P)

# def candidates(word): 
#     "Generate possible spelling corrections for word."
#     return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

# def known(words): 
#     "The subset of `words` that appear in the dictionary of WORDS."
#     return set(w for w in words if w in WORDS)

# def edits1(word):
#     "All edits that are one edit away from `word`."
#     letters    = 'abcdefghijklmnopqrstuvwxyz'
#     splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
#     deletes    = [L + R[1:]               for L, R in splits if R]
#     transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
#     replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
#     inserts    = [L + c + R               for L, R in splits for c in letters]
#     return set(deletes + transposes + replaces + inserts)

# def edits2(word): 
#     "All edits that are two edits away from `word`."
#     return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
# def spell_checker(text):
#     all_words = re.findall(r'\w+', text.lower()) # split sentence to words
#     spell_checked_text  = []
#     for i in range(len(all_words)):
#         spell_checked_text.append(correction(all_words[i]))
#     return ' '.join(spell_checked_text)

In [ ]:
# test_news['clean']

In [ ]:
# print("Before: \n", train_news['clean'][0] )
# t1 = time.time()
# train_news['clean'] = train_news['clean'].apply(spell_checker)
# t2 = time.time()
# print("\nTime to spell check the train data: \n", len(train_news), "news:", (t2-t1)/60, "min")

# print("\nAfter: \n",train_news['clean'][0] )
# train_news.head(5)

In [ ]:
# t1 = time.time()
# test_news['clean'] = test_news['clean'].apply(spell_checker)
# test_news.head(5)
# t2 = time.time()
# print("\nTime to spell check the test data: \n", len(test_news), "news:", (t2-t1)/60, "min")

In [ ]:
# t1 = time.time()
# valid_news['clean'] = valid_news['clean'].apply(spell_checker)
# valid_news.head(5)
# t2 = time.time()
# print("\nTime to spell check the valid data: \n", len(valid_news), "news:", (t2-t1)/60, "min")

In [ ]:
# train_news.to_csv("train_processed.csv", sep=',')
# test_news.to_csv("test_processed.csv", sep=',')
# valid_news.to_csv("valid_processed.csv", sep=',')

# Visualization 

#### WordCloud 

> As a tool for visualization by using the frequency of words appeared in text, we use WordCloud. Note that it can give more information and insight of texts by analyzing correlations and similarities between words rather than analyzing texts only by the frequency of words appeared; however, it can give you some general shape of what this text is about quickly and intuitively.

In [ ]:
# from wordcloud import WordCloud, STOPWORDS
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats

# %matplotlib inline

In [ ]:
# def cloud(data,backgroundcolor = 'white', width = 800, height = 600):
#     wordcloud = WordCloud(stopwords = STOPWORDS, background_color = backgroundcolor,
#                          width = width, height = height).generate(data)
#     plt.figure(figsize = (15, 10))
#     plt.imshow(wordcloud)
#     plt.axis("off")
#     plt.show()
    
# cloud(' '.join(train_news['clean']))

In [ ]:
#!conda install -c conda-forge xgboost

# Feature:Corpus Structure

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn import metrics

## creating a class of the feature "CorpusStructure" and defining the fucntions

In [ ]:
# creating a class of the feature "CorpusStructure" and defining the fucntions
# we need to use for 3 classifiers XGBoost, SVM, and Random Forest
class CorpusStructure():
    
    def __init__(self):

        self.xgboost_pipeline = ''
        self.svm_pipeline = ''
        self.logisticRegression_pipeline = ''
        self.randomForest_pipeline = ''
        
        self.xgboost_enrich_pipeline = ''
        self.svm_enrich_pipeline = ''
        self.logisticRegression_enrich_pipeline = ''
        self.randomForest_enrich_pipeline = ''
    
    # function of multi output label
    def multiOutputLabel(self, r):
        v = r['label']

        # catagorize 'true', 'mostly-true', 'half-true','barely-true', 'false', 'pants-fire'
        if (v == 'true'):
            return 5
        if (v == 'mostly-true'):
            return 4
        if (v == 'half-true'):
            return 3
        if (v == 'barely-true'):
            return 2
        if (v == 'false'):
            return 1
        if (v == 'pants-fire'):
            return 0

    # Piplines 
    # XGBoost Pipline ***************************************************
    def xgboost_pipline(self, ngram_input_range=(1,1)):
                
        tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=ngram_input_range)
        count_vectorizer = CountVectorizer(ngram_range=ngram_input_range)
        
        X = count_vectorizer.fit_transform(train_news['clean'])

        
        X_train = tfidf_vectorizer.fit_transform(train_news['clean'].values)
        X_test = tfidf_vectorizer.transform(test_news['clean'].values)
        
        train_news['multiLabel'] = train_news.apply(lambda row: self.multiOutputLabel(row), axis=1)
        test_news['multiLabel'] = test_news.apply(lambda row: self.multiOutputLabel(row), axis=1)

        y_train = train_news['multiLabel']
        y_test = test_news['multiLabel']

        # print("\ny_train:",y_train.shape)
        # print("y_train:",y_test.shape)
        # print("\ny_train:",y_train)
        # print("y_train:",y_test)
        
        self.xgboost_pipeline = Pipeline([
            ('vectorizer', tfidf_vectorizer),
            ('clf', xgb.XGBClassifier(random_state=1,learning_rate=0.01))
            ])


        self.xgboost_pipeline.fit(train_news['clean'].values, y_train)
        predicts = self.xgboost_pipeline.predict(test_news['clean'].values)
        score = metrics.accuracy_score(y_test, predicts)




        print("Accuracy:",score)
        print("\n")
        print(classification_report(y_test, predicts))
        return score

    # XGBoost Predict
    def xgboost_predict(self, text):
        predicted = self.xgboost_pipeline.predict([text])
        predicedProb = self.xgboost_pipeline.predict_proba([text])[:,1]
        return predicted, float(predicedProb)


    # SVM Pipline ***************************************************
    def svm_pipline(self, ngram_input_range=(1,1)):
                
        tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=ngram_input_range)
        count_vectorizer = CountVectorizer(ngram_range=ngram_input_range)
        
        X = count_vectorizer.fit_transform(train_news['clean'])

        
        X_train = tfidf_vectorizer.fit_transform(train_news['clean'].values)
        X_test = tfidf_vectorizer.transform(test_news['clean'].values)
        
        train_news['multiLabel'] = train_news.apply(lambda row: self.multiOutputLabel(row), axis=1)
        test_news['multiLabel'] = test_news.apply(lambda row: self.multiOutputLabel(row), axis=1)

        y_train = train_news['multiLabel']
        y_test = test_news['multiLabel']
        
        self.svm_pipeline = Pipeline([
            ('vectorizer', tfidf_vectorizer),
            ('clf', svm.SVC(probability=True))
            ])
        
        self.svm_pipeline.fit(train_news['clean'].values, y_train)
        predicts = self.svm_pipeline.predict(test_news['clean'].values)
        score = metrics.accuracy_score(y_test, predicts)

        print("Accuracy:",score)
        print("\n")
        print(classification_report(y_test, predicts))
        return score

    def svm_predict(self, text):
        predicted = self.svm_pipeline.predict([text])
        predicedProb = self.svm_pipeline.predict_proba([text])[:,1]
        return predicted, float(predicedProb)

    
    # Random Forest Pipline ***************************************************
    def rf_pipline(self, ngram_input_range=(1,1)):
                
        tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=ngram_input_range)
        count_vectorizer = CountVectorizer(ngram_range=ngram_input_range)
        
        X = count_vectorizer.fit_transform(train_news['clean'])

        
        X_train = tfidf_vectorizer.fit_transform(train_news['clean'].values)
        X_test = tfidf_vectorizer.transform(test_news['clean'].values)
        
        train_news['multiLabel'] = train_news.apply(lambda row: self.multiOutputLabel(row), axis=1)
        test_news['multiLabel'] = test_news.apply(lambda row: self.multiOutputLabel(row), axis=1) 

        y_train = train_news['multiLabel']
        y_test = test_news['multiLabel']
        
        self.randomForest_pipeline = Pipeline([
            ('vectorizer', tfidf_vectorizer),
            ('clf', RandomForestClassifier(n_estimators = 300))
            ])




        self.randomForest_pipeline.fit(train_news['clean'].values, y_train)
        predicts = self.randomForest_pipeline.predict(test_news['clean'].values)
        score = metrics.accuracy_score(y_test, predicts)
        print("Accuracy:",score)
        print("\n")
        print(classification_report(y_test, predicts))
        return score

    def rf_predict(self, text):
        predicted = self.randomForest_pipeline.predict([text])
        predicedProb = self.randomForest_pipeline.predict_proba([text])[:,1]
        return predicted, float(predicedProb)


    # data enrichment with another dataset   *****************************
    # Content articles1.csv - 50,000 news articles (Articles 1-50,000)
    def data_enrichment(self):

        df_articles = pd.read_csv("articles1.csv")
        df_articles.head(3)
        
        publications = np.unique(df_articles["publication"].values)
        print("publications:\n")
        print(publications)
        
        df_articles = df_articles.loc[df_articles['publication'].isin(['New York Times', 'CNN','Atlantic','Fox News','Guardian','National Review','NPR','Reuters', 'Vox','Washington Post'])]

        df_articles.to_csv('real_news.csv', encoding='utf-8')

        fake_news = pd.read_csv("fake.csv")
        # fake_news = fake_news[:1000]   
        real_news = pd.read_csv("real_news.csv")
        # real_news = real_news[:1000]  

        
        real_news2 = real_news[['title', 'content', 'publication']]
        real_news2.dropna()
        real_news2['multiLabel'] = 1
        real_news2.head(3)
    
        fake_news2 = fake_news[['title', 'text','site_url']]
        fake_news2.dropna()
        fake_news2['multiLabel'] = 0
        fake_news2.head(5)
        
        # cleaning the data and add it as a column of the dataset 
        fake_news2['clean'] = fake_news2['text'].apply(cleaning)
        real_news2['clean'] = real_news2['content'].apply(cleaning)
        print("After cleaning\n")
        fake_news2.head(3)
        real_news2.head(3)    
        
        # concatenate real and fake news data together
        data_news = [real_news2, fake_news2]
        df_news = pd.concat(data_news)[['clean', 'multiLabel']]
        # train, test split
        df_train, df_test = train_test_split(df_news, test_size=0.2, random_state=0)
        
        # Enrich with original data frame
        df_news_enriched = [df_train, train_news]
        news_dataset = pd.concat(df_news_enriched)[['clean', 'multiLabel']]

        

        
        X_test_enrich = [test_news, df_test]
        df_test_enrich = pd.concat(X_test_enrich)[['clean', 'multiLabel']]

        
        y_test_enrich = [test_news['multiLabel'], df_test['multiLabel']]
        df_y_test_enrich = pd.concat(y_test_enrich)
        df_y_test_enrich
        
        return news_dataset, df_test_enrich, df_y_test_enrich;

    # XGBoost enrich pipline  ***************************
    def xgboost_enrich_pipline(self, ngram_input_range=(1,1)):
        
        news_dataset, df_test_enrich, df_y_test_enrich = self.data_enrichment()

        tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=ngram_input_range)

        self.xgboost_enrich_pipeline = Pipeline([
            ('vectorizer', tfidf_vectorizer),
            ('clf', xgb.XGBClassifier(random_state=1,learning_rate=0.01))
            ])

        self.xgboost_enrich_pipeline.fit(news_dataset['clean'].values, news_dataset['multiLabel'])
        predicts = self.xgboost_enrich_pipeline.predict(df_test_enrich['clean'].values)
        score = metrics.accuracy_score(df_y_test_enrich, predicts)

        print("Accuracy:",score)
        print("\n")
        print(classification_report(df_y_test_enrich, predicts))
        return score


    # SVM enrich pipline  ***************************
    def svm_enrich_pipline(self, ngram_input_range=(1,1)):
        
        news_dataset, df_test_enrich, df_y_test_enrich = self.data_enrichment()

        tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=ngram_input_range)

        self.svm_enrich_pipeline = Pipeline([
            ('vectorizer', tfidf_vectorizer),
            ('clf', svm.SVC(probability=True))
            ])

        self.svm_enrich_pipeline.fit(news_dataset['clean'].values, news_dataset['multiLabel'])
        predicts = self.svm_enrich_pipeline.predict(df_test_enrich['clean'].values)
        score = metrics.accuracy_score(df_y_test_enrich, predicts)

        print("Accuracy:",score)
        print("\n")
        print(classification_report(df_y_test_enrich, predicts))
        return score
        
    # Random Forest enrich pipline  ***************************    
    def rf_enrich_pipline(self, ngram_input_range=(1,1)):
        
        news_dataset, df_test_enrich, df_y_test_enrich = self.data_enrichment()

        tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=ngram_input_range)

        self.randomForest_enrich_pipeline = Pipeline([
            ('vectorizer', tfidf_vectorizer),
            ('clf', RandomForestClassifier(n_estimators = 300))
            ])

        self.randomForest_enrich_pipeline.fit(news_dataset['clean'].values, news_dataset['multiLabel'])
        predicts = self.randomForest_enrich_pipeline.predict(df_test_enrich['clean'].values)
        score = metrics.accuracy_score(df_y_test_enrich, predicts)
        print("Accuracy:",score)
        print("\n")
        print(classification_report(df_y_test_enrich, predicts))
        return score


    #predict based on XGBoost
    def xgboost_enrich_predict(self, text):
        predicted = self.xgboost_enrich_pipeline.predict([text])
        predicedProb = self.xgboost_enrich_pipeline.predict_proba([text])[:,1]
        return predicted, float(predicedProb)

    #predict based on SVM
    def svm_enrich_predict(self, text):
        predicted = self.svm_enrich_pipeline.predict([text])
        predicedProb = self.svm_enrich_pipeline.predict_proba([text])[:,1]
        return predicted, float(predicedProb)


    #predict based on Random Forest
    def rf_enrich_predict(self, text):
        predicted = self.randomForest_enrich_pipeline.predict([text])
        predicedProb = self.randomForest_enrich_pipeline.predict_proba([text])[:,1]
        
        return predicted, float(predicedProb)

# 1. What are the biggest impact features for your factor? (gini index) etc. using liar liar, and adding a second dataset.


## Creating an object of CorpusStructure 

In [ ]:
corpusStructure = CorpusStructure()

In [ ]:
accuracy_dictionary_2gram = {}
accuracy_dictionary_3gram = {}
accuracy_dictionary_4gram = {}

# 2. compare : with columns 1-x, here are the algorithm score, adding the rest of columns in liarliar brings us to / changes to this f1, accuracy, roc/auc, polynomial

In [ ]:
#Uni-gram
xgb_accuracy_2gram = corpusStructure.xgboost_pipline(ngram_input_range=(1,1))


In [ ]:
#Uni-gram
svm_accuracy_2gram = corpusStructure.svm_pipline(ngram_input_range=(1,1))

In [ ]:
#Uni-gram
rfc_accuracy_2gram = corpusStructure.rf_pipline(ngram_input_range=(1,1))

## Bi-gram

In [ ]:
xgb_accuracy_2gram = corpusStructure.xgboost_pipline(ngram_input_range=(2,2))
accuracy_dictionary_2gram['XGB'] = xgb_accuracy_2gram

In [ ]:
svm_accuracy_2gram = corpusStructure.svm_pipline(ngram_input_range=(2,2))
accuracy_dictionary_2gram['SVM'] = svm_accuracy_2gram

In [ ]:
rfc_accuracy_2gram = corpusStructure.rf_pipline(ngram_input_range=(2,2))
accuracy_dictionary_2gram['RF'] = rfc_accuracy_2gram

## Tri-gram

In [ ]:
xgb_accuracy_3gram = corpusStructure.xgboost_pipline(ngram_input_range=(3,3))
accuracy_dictionary_3gram['XGB'] = xgb_accuracy_3gram

In [ ]:
svm_accuracy_3gram = corpusStructure.svm_pipline(ngram_input_range=(3,3))
accuracy_dictionary_3gram['SVM'] = svm_accuracy_3gram

In [ ]:
rfc_accuracy_3gram = corpusStructure.rf_pipline(ngram_input_range=(3,3))
accuracy_dictionary_3gram['RF'] = rfc_accuracy_3gram

## 4-gram

In [ ]:
xgb_accuracy_4gram = corpusStructure.xgboost_pipline(ngram_input_range=(4,4))
accuracy_dictionary_4gram['XGB'] = xgb_accuracy_4gram

In [ ]:
svm_accuracy_4gram = corpusStructure.svm_pipline(ngram_input_range=(4,4))
accuracy_dictionary_4gram['SVM'] = svm_accuracy_4gram

In [ ]:
rfc_accuracy_4gram = corpusStructure.rf_pipline(ngram_input_range=(4,4))
accuracy_dictionary_4gram['RF'] = rfc_accuracy_4gram

## Comparing the results

In [ ]:
import matplotlib.pylab as plt
#plt.bar(accuracy_dictionary.keys(), accuracy_dictionary.values(), color='g')
#print(accuracy_dictionary_2gram)

In [ ]:
lists2 = sorted(accuracy_dictionary_2gram.items()) # sorted by key, return a list of tuples
lists3 = sorted(accuracy_dictionary_3gram.items()) # sorted by key, return a list of tuples
lists4 = sorted(accuracy_dictionary_4gram.items()) # sorted by key, return a list of tuples


x2, y2 = zip(*lists2) # unpack a list of pairs into two tuples
x3, y3 = zip(*lists3) # unpack a list of pairs into two tuples
x4, y4 = zip(*lists4) # unpack a list of pairs into two tuples

plt.plot(x2, y2, label ='Bi-gram', color='red')
plt.plot(x3, y3, label ='tri-gram', color='blue')
plt.plot(x4, y4, label = '4-gram', color='green')
plt.legend(loc='best')
plt.show()

## Data Amalgamation - Enriching the models

In [ ]:
accuracy_dictionary_enriched = {}

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
xgb_accuracy_enriched = corpusStructure.xgboost_enrich_pipline(ngram_input_range=(4,4))
accuracy_dictionary_enriched['XGB'] = xgb_accuracy_enriched

In [ ]:
svm_accuracy_enriched = corpusStructure.svm_enrich_pipline(ngram_input_range=(4,4))
accuracy_dictionary_enriched['SVM'] = svm_accuracy_enriched

In [ ]:
rfc_accuracy_enriched = corpusStructure.rf_enrich_pipline(ngram_input_range=(4,4))
accuracy_dictionary_enriched['RF'] = rfc_accuracy_enriched

## Comparing the results after enriching

In [ ]:
import matplotlib.pylab as plt
#plt.bar(accuracy_dictionary.keys(), accuracy_dictionary.values(), color='g')
#print(accuracy_dictionary_2gram)

In [ ]:
print(accuracy_dictionary_enriched)

In [ ]:
lists_enriched = sorted(accuracy_dictionary_enriched.items()) # sorted by key, return a list of tuples

x_enriched, y_enriched = zip(*lists_enriched) # unpack a list of pairs into two tuples

plt.plot(x_enriched, y_enriched, label ='4-gram Enriched', color='red')
plt.legend(loc='best')
plt.show()

# 3. compare with and without amalgamation : how does it affect confusion matrix and metrics



 Data amalgamation increases the accuracy for example for the XGBoost model the acruracy increases from 20% to 54%. The table below shows the differences with and without ammalgametion:

 ### comparing the accuracy with and without amalgamation for 4-gram:

|Accuracy  |  Without Amalgamation |  With Amalgamation |
|-----|-----|-----|
| XGBoost                 |  0.2123 |  0.5429 |  
| SVM                 |  0.2036 |0.6640 |
| Random Forest  |  0.203  |0.6634  |


 


# 5. with only liar liar --> then derived /augmented columns --> then dataset2 --> then dataset3 : compare at each step and show how results improve (or not) 
Dataset 1: Liar liar
Dataset 2: "fake.csv"
Dataset 3: "articles1.csv"



# Get Corpus Structure Score

In [ ]:
def DATAMINERS_getCorpusStructureScore_xgboost(text): 
    multiLabelValue, probValue = corpusStructure.xgboost_enrich_predict(text)
    
    return multiLabelValue[0], 1 - float(probValue)

In [ ]:
def DATAMINERS_getCorpusStructureScore_svm(text): 
    multiLabelValue, probValue = corpusStructure.svm_enrich_predict(text)
    return multiLabelValue[0], 1 - float(probValue)

In [ ]:
def DATAMINERS_getCorpusStructureScore_rf(text): 
    multiLabelValue, probValue = corpusStructure.rf_enrich_predict(text)
    return multiLabelValue[0], 1 - float(probValue)

In [ ]:
print(DATAMINERS_getCorpusStructureScore_xgboost("McCain opposed a requirement that the government buy American-made motorcycles. And he said all buy-American provisions were quote 'disgraceful."))
## catagorize 'true'= 5, 'mostly-true'= 4, 'half-true'= 3,'barely-true'=2, 'false'=1, 'pants-fire'=0

In [ ]:
# print(DATAMINERS_getCorpusStructureScore_xgboost("When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration."))

In [ ]:
#!pip install transformers

In [ ]:
#type(corpusStructure.randomForest_enrich_pipeline)

#6. How can we label the bs column and distribute with other labels in the kaggle fake news dataset? What works and what did not work? <balancing text based data set labels>



In [ ]:
# # Fake News Dataset column/feature names
fake_labels = ['uuid', 'ord_in_thread', 'author', 'published', 'title', 'text', 'language', 'crawled',
            'site_url', 'country', 'domain_rank', 'thread_title', 'spam_score', 'main_img_url',
            'replies_count', 'participants_count', 'likes', 'comments', 'shares', 'type']

# Fake News Dataset passed to a daraframe
#fake_bias_data.GetContentFile('fake.csv')  
fake_dataset = pd.read_csv('fake.csv', error_bad_lines=False)
output = fake_dataset.to_csv('fake.csv')

In [ ]:
fake_dataset.head(-10)

In [ ]:
fake_dataset.info()

In [ ]:
fake_dataset['type'].value_counts().plot(kind = 'bar')

In [ ]:
fake_dataset['type'].value_counts()

In [ ]:
fake_dataset['type'].describe()

In [ ]:
fake_dataset['type'].value_counts(normalize=True)

In [ ]:
fake_dataset['text']

# 7. Polynomial equation for multi-class classification : try diff methods : average, weighted average, transformer.
The following cells are trying transformers




# 10. Use a Transformer in your Factor, eg, multi-class classification, or weights of attentions in your polynomial instead of accuracies.
The following cells are trying transformers


Trying Transformers

## Importing the dataset


In [ ]:
#df = pd.read_csv('train.tsv', delimiter='\t', header=None)

In [ ]:
#df.shape

In [ ]:
#train_news.shape

In [ ]:
df = train_news

In [ ]:
batch_1 = df
#batch_1 = train_news

We can ask pandas how many sentences are labeled as "false", "mostly-true", "half-true", "true", "barely true" and "pants-fire"

In [ ]:
batch_1['label'].value_counts()

In [ ]:
!pip install transformers
!pip install torchvision 

!pip install tensorflow-gpu

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
#from transformers import *
from transformers import DistilBertTokenizer, RobertaTokenizer 


## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
tokenized = batch_1['headline_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

### Padding
After tokenization, `tokenized` is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [ ]:
np.array(padded).shape

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

## Model #1: And Now, Deep Learning!
The model and inputs are ready, running the model!

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = batch_1[1]

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

In [ ]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [ ]:
# lr_clf = LogisticRegression()
# lr_clf.fit(train_features, train_labels)

In [ ]:
xgb_clf = XGBClassifier()
xgb_clf.fit(train_features, train_labels)
y_pred_xgb = xgb_clf.predict(test_features)
print("Accuracy:", xgb_clf.score(test_features, test_labels))
print(classification_report(test_labels,y_pred_xgb))

In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(train_features, train_labels)
y_pred_rfc = rf_clf.predict(test_features)
print("Accuracy:", rf_clf.score(test_features, test_labels))
print(classification_report(test_labels,y_pred_rfc))

In [ ]:
svm_clf = svm.SVC()
svm_clf.fit(train_features, train_labels)
y_pred_svm = svm_clf.predict(test_features)
print("Accuracy:", svm_clf.score(test_features, test_labels))
print(classification_report(test_labels,y_pred_svm))

 ### comparing the accuracy with and without transforming:

|Accuracy  |  Without Transforming |  With Transforming |
|-----|-----|-----|
| XGBoost                 |  0.2123 |  0.252 |  
| SVM                 |  0.2036 |0.26 |
| Random Forest  |  0.203  |0.268  |


# 9. What are your conclusions on how can we improve multi-class classification on the liar liar dataset , what methods work to increase scores/metrics? (edited) 



# 11 Extra credit: Implement a Topic Naming Solution on one of the LDAs from your dataset 



In [ ]:
#LDA

In [ ]:
train_news['index'] = train_news.index
data = train_news
train_lda = data[['clean', 'index']]
train_lda.head(4)

In [ ]:
test_news['index'] = test_news.index
data = test_news
test_lda = data[['clean', 'index']]
test_lda.head(4)

In [ ]:
valid_news['index'] = valid_news.index
data = valid_news
valid_lda = data[['clean', 'index']]
valid_lda.head(4)

In [ ]:
processed_docs = train_lda['clean'].map(lambda doc: doc.split(" "))
processed_docs[:4]

In [ ]:
import gensim
def get_word_tokens(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if len(token) > 3:
            result.append(token)
    return result
tokenized_docs_local = train_news['clean'].map(get_word_tokens)

In [ ]:
def get_dictionary_print_words(dataframe,colname):
    dictionary_gensim = gensim.corpora.Dictionary(processed_docs)
    count = 0
    print('######## DICTIONARY Words and occurences ########')
    for k, v in dictionary_gensim.iteritems():
        print(k, v)
        count += 1
        if count > 10:
            break
    dictionary_gensim.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    return dictionary_gensim, tokenized_docs_local

In [ ]:
def get_bow_corpus_print_sample(dataframe,colname):
    dictionary_gensim, tokenized_docs_local = get_dictionary_print_words(dataframe, colname)
    bow_corpus_local = [dictionary_gensim.doc2bow(doc) for doc in tokenized_docs_local]
    bow_doc_local_0 = bow_corpus_local[0]
    print('\n ######## BOW VECTOR FIRST ITEM ########')
    print(bow_doc_local_0)
    print('\n ######## PREVIEW BOW ########')
    for i in range(len(bow_doc_local_0)):
        print("Word {} (\"{}\") appears {} time.".format(bow_doc_local_0[i][0], 
                                               dictionary_gensim[bow_doc_local_0[i][0]], bow_doc_local_0[i][1]))
    return bow_corpus_local, dictionary_gensim

In [ ]:
def get_tfidf_corpus_print_sample(bow_corpus_local):
    from gensim import corpora, models
    tfidf = models.TfidfModel(bow_corpus_local)
    tfidf_corpus_local = tfidf[bow_corpus_local]
    print('\n ######## TFIDF VECTOR FIRST ITEM ########')
    
    from pprint import pprint
    for doc in tfidf_corpus_local:
        pprint(doc)
        break
    return tfidf_corpus_local

## Running ldamodel 

In [ ]:
def get_lda_model_print_top_topics(bow_corpusforlda,numtopics,dictionaryforlda):
    lda_model = gensim.models.LdaMulticore(bow_corpusforlda, num_topics=numtopics, id2word=dictionaryforlda, passes=2, workers=2)
    lda_all_topics=lda_model.show_topics(num_topics=numtopics, num_words=10,formatted=False)
    lda_topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in lda_all_topics]

    #Below Code Prints Topics and Words
    for topic,words in lda_topics_words:
        print(str(topic)+ "::"+ str(words))
    return lda_model

In [ ]:
def get_lda_model_topics_topwords_print_top_topics(bow_corpusforlda,numtopics,dictionaryforlda):
    lda_model = gensim.models.LdaMulticore(bow_corpusforlda, num_topics=numtopics, id2word=dictionaryforlda, passes=2, workers=2, random_state=1)
    lda_all_topics=lda_model.show_topics(num_topics=numtopics, num_words=10,formatted=False)
    lda_topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in lda_all_topics]

    #Below Code Prints Topics and Words
    for topic,words in lda_topics_words:
        print(str(topic)+ "::"+ str(words))
    return lda_model,lda_topics_words

In [ ]:
def identify_topic_number_score_label_topwords(text,dictionary_local,lda_model_local,lda_topics_top_words_local):
    bow_vector_local = dictionary_local.doc2bow(get_word_tokens(text))
    topic_number_local, topic_score_local = sorted(
        lda_model_local[bow_vector_local], key=lambda tup: -1*tup[1])[0]
    #print (topic_number_local, topic_score_local)
    return pd.Series([topic_number_local, topic_score_local," ".join(lda_topics_top_words_local[int(topic_number_local)][1])])

In [ ]:
def update_lda_results_to_dataset(dataframe,topiccolnames,coltoapplylda,colnamedictionary,colnameldamodel, colnameldatopwords):
    dataframe[topiccolnames] = dataframe.apply(
    lambda row: identify_topic_number_score_label_topwords(
        row[coltoapplylda],colnamedictionary,colnameldamodel,
        colnameldatopwords), axis=1)
    return dataframe

## Creating Bag of Words

In [ ]:
bow_corpus_headline, dictionary_headline = get_bow_corpus_print_sample(train_news,
                                                                      'clean')

In [ ]:
lda_model_headline, lda_headline_topic_words = get_lda_model_topics_topwords_print_top_topics(bow_corpus_headline, 10 ,dictionary_headline)

Generating TF-IDF bow_corpus:

In [ ]:
tfidf_corpus_headline = get_tfidf_corpus_print_sample(bow_corpus_headline)

In [ ]:
lda_tfidf_model_headline  = get_lda_model_print_top_topics(tfidf_corpus_headline,10,dictionary_headline)

In [ ]:
semisupervised_topic_labels = ['topic0','topic1','topic2','topic3','topic4','topic5','topic6','topic7','topic8','topic9']

In [ ]:
headlinetopiccolnames = ['topic_number','lda_score','topic_top_words']
train_news = update_lda_results_to_dataset(
    train_news, headlinetopiccolnames,'clean', dictionary_headline, lda_model_headline, lda_headline_topic_words)
train_news.head()

In [ ]:
test_news = update_lda_results_to_dataset(
    test_news,headlinetopiccolnames,'clean',
  dictionary_headline,lda_model_headline,lda_headline_topic_words)
test_news.head()

In [ ]:
import seaborn as sb
def create_distribution(dataFile):
    g = sb.countplot(x='topic_number', data=dataFile, palette='hls')
    g.set_xticklabels(g.get_xticklabels(),rotation=90)

    return g

create_distribution(train_news) # TRAIN Document Vector

In [ ]:
create_distribution(test_news)# TEST Document Vector

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',  
                            max_features=50000,)
data_vectorized = vectorizer.fit_transform(train_news['clean'])
# apply transformation
tf = vectorizer.fit_transform(train_news['clean']).toarray()
# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

number_of_topics = 10

lda_model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)
lda_model.fit(tf)
# Best Model
#lda_model = model.best_estimator_
# Model Parameters
#print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
#print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", lda_model.perplexity(data_vectorized))

In [ ]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)
no_top_words = 10
display_topics(lda_model, tf_feature_names, no_top_words)

In [ ]:
# Create Document — Topic Matrix
lda_output = lda_model.transform(data_vectorized)
# column names
topicnames = ['Topic' + str(i) for i in range(lda_model.n_components)]
# index names
docnames = [i for i in range(len(train_news))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics


In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

In [ ]:
combined_train = train_news.drop(['clean','headline_text','jsonid','subject',	'speaker',	'speakerjobtitle',	'stateinfo',	'partyaffiliation','context'], axis=1)


In [ ]:
# Change Dataset values to numeric
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder() 
combined_train['label'] = lb.fit_transform(combined_train['label'])
combined_train['topic_top_words'] = lb.fit_transform(combined_train['topic_top_words'])


In [ ]:
from sklearn.model_selection import train_test_split


lda = pd.merge(combined_train,df_document_topic,left_index =True ,right_index=True)
X_train, X_test, y_train, y_test = train_test_split(lda.drop(['label'], axis=1),lda['label'], test_size=0.33, random_state=42)

In [ ]:
def run_models(X_train, y_train, X_test, y_test):
    models = []
    models.append(('RandomForest', RandomForestClassifier(n_estimators=10)))
    models.append(('SVM', SVC(gamma='auto')))
    models.append(('XGBoost', XGBClassifier(random_state=1,learning_rate=0.01, early_stopping_rounds=10)))
    for name, model in models:
        model.fit(X_train,y_train)
        print(name, ': ')
        # make predictions for test data
        y_pred = model.predict(X_test)
        predictions = [round(value) for value in y_pred]

        # evaluate predictions
        from sklearn.metrics import classification_report
        print(classification_report(y_test,predictions))
    


In [ ]:
# scale data using StandardScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC

# scaler = StandardScaler()
# lda_scaled_train = scaler.fit_transform(X_train)
# lda_scaled_test = scaler.fit_transform(X_test)
# lda_scaled = scaler.fit_transform(lda.drop(['label'], axis=1))
# run_models(lda_scaled_train,y_train, lda_scaled_test,y_test)

In [ ]:
xgb_clf = XGBClassifier()
eval_set = [(X_test, y_test)]
xgb_clf.fit(X_train, y_train, early_stopping_rounds=10, eval_set=eval_set)
y_pred_xgb = xgb_clf.predict(X_test)
print("Accuracy:", xgb_clf.score(X_test, y_test))
print(classification_report(y_test,y_pred_xgb))

In [ ]:
rf_clf = RandomForestClassifier()
# X_test = X_test.fillna(X_train.mean())
# X_train = X_test.fillna(X_train.mean())
rf_clf.fit(X_train, y_train)
y_pred_rfc = rf_clf.predict(X_test)
print("Accuracy:", rf_clf.score(X_test, y_test))
print(classification_report(y_test,y_pred_rfc))

In [ ]:
svm_clf = svm.SVC()
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("Accuracy:", svm_clf.score(X_test, y_test))
print(classification_report(y_test,y_pred_svm))

 ### comparing the accuracy with and without LDA:

|Accuracy  |  Without LDA |  With LDA |
|-----|-----|-----|
| XGBoost                 |  0.2123 |  0.446 |  
| SVM                 |  0.2036 |0.218 |
| Random Forest  |  0.203  |0.425  |


In [ ]:
# unexpected result for svm after LDA

 ### Concluding the comparision of the accuracy with and without different situations:


|Accuracy  |  Without Anythnig |  With Transforming |  With LDA |  With Amalgamation |
|-----|-----|-----|-----|-----|
| XGBoost                 |  0.2123 |  0.252 |  0.446 |    0.5429 |
| SVM                 |  0.2036 |0.26 |0.218 |0.6640 |
| Random Forest  |  0.203  |0.268  |0.425  |0.6634  |

